In [1]:
# Data download and unzipping
!wget https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_test.zip -c
!wget https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_train.zip -c

!mkdir stage1_train stage1_test

!unzip -q stage1_train.zip -d stage1_train/
!unzip -q stage1_test.zip -d stage1_test/

--2018-03-27 22:49:32--  https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_test.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9545388 (9.1M) [application/zip]
Saving to: ‘stage1_test.zip’

stage1_test.zip     100%[===================>]   9.10M  18.1MB/s    in 0.5s    

2018-03-27 22:49:32 (18.1 MB/s) - ‘stage1_test.zip’ saved [9545388/9545388]

--2018-03-27 22:49:33--  https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP requ

In [2]:
# Downloading and intstalling keras
!pip install keras
!pip install tqdm

# Downloading the library Mask_RCNN
# !git clone https://github.com/matterport/Mask_RCNN.git
!git clone https://github.com/ZengChen94/Mask_RCNN.git
!mv ./Mask_RCNN/* ./
!rm -r ./Mask_RCNN
!ls

Cloning into 'Mask_RCNN'...
remote: Counting objects: 335, done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 335 (delta 7), reused 14 (delta 6), pack-reused 319
Receiving objects: 100% (335/335), 50.52 MiB | 40.01 MiB/s, done.
Resolving deltas: 100% (176/176), done.
mv: cannot move './Mask_RCNN/assets' to './assets': Directory not empty
mv: cannot move './Mask_RCNN/images' to './images': Directory not empty
mv: cannot move './Mask_RCNN/samples' to './samples': Directory not empty
assets		    inspect_model.ipynb    __pycache__	    stage1_train.zip
coco.py		    inspect_weights.ipynb  README.md	    train_shapes.ipynb
config.py	    LICENSE		   samples	    utils.py
datalab		    logs		   shapes.py	    visualize.py
demo.ipynb	    mask_rcnn_coco.h5	   stage1_test
images		    model.py		   stage1_test.zip
inspect_data.ipynb  parallel_model.py	   stage1_train


In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline 

In [4]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
import utils
import cv2
from glob import glob
import skimage.io
import seaborn as sns
import sys
import math
import re
import time
import matplotlib
import matplotlib.patches as patches
from config import Config
import model as modellib
import visualize
from model import log

ROOT_DIR = os.getcwd()
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

Using TensorFlow backend.


In [0]:
TRAIN_PATH = './stage1_train/'
TEST_PATH = './stage1_test/'
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]
kernel = np.ones((1,1),np.uint8)

In [0]:
class KaggleTrainDataset(utils.Dataset):
    def get_path(self, id_):
        return './stage1_train/' + id_ + '/images/' + id_ + '.png'
    
    def load_data(self, train_ids):
        self.add_class("nuclei", 1, "nuclei")
        self.sizes = []
        self.img_ids = train_ids
        for i, id_ in tqdm(enumerate(self.img_ids), total=len(self.img_ids)):
            path = self.get_path(id_)
            img = imread(path)[:, :, :3]
            img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
            w, h = img.shape[0], img.shape[1]
            self.sizes.append((w, h))
            self.add_image("nuclei", image_id = i, path=id_, width=w, height=h)

    def load_image(self, image_id):
        image_name = self.img_ids[image_id]
        path = self.get_path(image_name)
        img = imread(path)[:,:,:3]
        w, h = img.shape[0], img.shape[1]
        return img
    
    def load_mask(self, image_id):
        id_ = self.img_ids[image_id]
        mask_path = './stage1_train/' + id_ + '/masks/'
        num_masks = len(next(os.walk(mask_path))[2])
        width, height = self.sizes[image_id]
        mask = np.zeros((width, height, num_masks), dtype=np.bool)
        for i, mask_file in enumerate(next(os.walk(mask_path))[2]):
            mask_ = imread(mask_path + mask_file)
            mask_ = mask_.reshape((mask_.shape[0], mask_.shape[1], 1))
            mask[:,:,i:i+1] = mask_
        return mask, np.ones(num_masks, np.int32)
      
class KaggleTestDataset(utils.Dataset):
    def get_path(self, id_):
        return './stage1_test/' + id_ + '/images/' + id_ + '.png'
    
    def load_data(self, test_ids):
        self.add_class("nuclei", 1, "nuclei")
        self.sizes = []
        self.img_ids = test_ids
        for i, id_ in tqdm(enumerate(self.img_ids), total=len(self.img_ids)):
            path = self.get_path(id_)
            img = imread(path)[:, :, :3]
            img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
            w, h = img.shape[0], img.shape[1]
            self.sizes.append((w, h))
            self.add_image("nuclei", image_id = i, path=id_, width=w, height=h)

    def load_image(self, image_id):
        image_name = self.img_ids[image_id]
        path = self.get_path(image_name)
        img = imread(path)[:,:,:3]
        w, h = img.shape[0], img.shape[1]
        return img

In [7]:
# Load dataset
dataset_train = KaggleTrainDataset()
dataset_train.load_data(train_ids[:600])
dataset_train.prepare()

dataset_val = KaggleTrainDataset()
dataset_val.load_data(train_ids[600:670])
dataset_val.prepare()

dataset_test = KaggleTestDataset()
dataset_test.load_data(test_ids)
dataset_test.prepare()

100%|██████████| 65/65 [00:00<00:00, 108.41it/s]


In [8]:
# Configurations
class KaggleConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    NAME = "shapes"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8
    NUM_CLASSES = 1 + 1
    IMAGE_MIN_DIM = 256*2
    IMAGE_MAX_DIM = 256*2
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  
    TRAIN_ROIS_PER_IMAGE = 32
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 5
config = KaggleConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_SHAPES                [[128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]
 [  8   8]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     8
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 8
IMAGE_MAX_DIM                  512
IMAGE_MIN_DIM                  512
IMAGE_PADDING                  True
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           shapes
NUM_CLASSES                    2
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE  

In [0]:
# Notebook Preferences
def get_ax(rows=1, cols=1, size=8):
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [0]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

# Weights to start with
init_with = "coco"  # imagenet, coco, or last
if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)

In [0]:
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# # Train the head branches
# # Passing layers="heads" freezes all layers except the head
# # layers. You can also pass a regular expression to select
# # which layers to train by name pattern.
# model.train(dataset_train, dataset_val, 
#             learning_rate=config.LEARNING_RATE, 
#             epochs=1, layers='heads')

In [11]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# model_path = model.find_last()[1]
# model.load_weights(model_path, by_name=True)

# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=50, 
            layers="all")


Starting at epoch 0. LR=0.0001

Checkpoint Path: /content/logs/shapes20180328T0118/mask_rcnn_shapes_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         (Conv2D)
bn3a_branch2b         

/usr/local/lib/python3.6/dist-packages/scipy/ndimage/interpolation.py:600: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/50
100/100 [==============================] - 467s 5s/step - loss: 3.4189 - rpn_class_loss: 0.6386 - rpn_bbox_loss: 1.1403 - mrcnn_class_loss: 0.5471 - mrcnn_bbox_loss: 0.7015 - mrcnn_mask_loss: 0.3915 - val_loss: 1.9328 - val_rpn_class_loss: 0.1468 - val_rpn_bbox_loss: 0.8797 - val_mrcnn_class_loss: 0.1918 - val_mrcnn_bbox_loss: 0.3850 - val_mrcnn_mask_loss: 0.3295
Epoch 2/50
100/100 [==============================] - 397s 4s/step - loss: 1.7559 - rpn_class_loss: 0.1919 - rpn_bbox_loss: 0.8169 - mrcnn_class_loss: 0.1891 - mrcnn_bbox_loss: 0.2729 - mrcnn_mask_loss: 0.2852 - val_loss: 1.7525 - val_rpn_class_loss: 0.1057 - val_rpn_bbox_loss: 0.8332 - val_mrcnn_class_loss: 0.2478 - val_mrcnn_bbox_loss: 0.3018 - val_mrcnn_mask_loss: 0.2640
Epoch 3/50
 20/100 [=====>........................] - ETA: 5:09 - loss: 1.6352 - rpn_class_loss: 0.1648 - rpn_bbox_loss: 0.7827 - mrcnn_class_loss: 0.1844 - mrcnn_bbox_loss: 0.2400 - mrcnn_mask_loss: 0.2633

100/100 [==============================] - 392s 4s/step - loss: 1.5865 - rpn_class_loss: 0.1483 - rpn_bbox_loss: 0.7649 - mrcnn_class_loss: 0.1814 - mrcnn_bbox_loss: 0.2303 - mrcnn_mask_loss: 0.2616 - val_loss: 1.6263 - val_rpn_class_loss: 0.0904 - val_rpn_bbox_loss: 0.8147 - val_mrcnn_class_loss: 0.1631 - val_mrcnn_bbox_loss: 0.2805 - val_mrcnn_mask_loss: 0.2776
Epoch 4/50
100/100 [==============================] - 394s 4s/step - loss: 1.4788 - rpn_class_loss: 0.1161 - rpn_bbox_loss: 0.7240 - mrcnn_class_loss: 0.1753 - mrcnn_bbox_loss: 0.2114 - mrcnn_mask_loss: 0.2520 - val_loss: 1.5628 - val_rpn_class_loss: 0.0798 - val_rpn_bbox_loss: 0.7917 - val_mrcnn_class_loss: 0.1477 - val_mrcnn_bbox_loss: 0.2693 - val_mrcnn_mask_loss: 0.2742
Epoch 5/50
 26/100 [======>.......................] - ETA: 4:51 - loss: 1.4056 - rpn_class_loss: 0.1000 - rpn_bbox_loss: 0.7122 - mrcnn_class_loss: 0.1667 - mrcnn_bbox_loss: 0.1921 - mrcnn_mask_loss: 0.2346

100/100 [==============================] - 395s 4s/step - loss: 1.4058 - rpn_class_loss: 0.1008 - rpn_bbox_loss: 0.7010 - mrcnn_class_loss: 0.1693 - mrcnn_bbox_loss: 0.1972 - mrcnn_mask_loss: 0.2375 - val_loss: 1.4830 - val_rpn_class_loss: 0.0737 - val_rpn_bbox_loss: 0.7747 - val_mrcnn_class_loss: 0.1301 - val_mrcnn_bbox_loss: 0.2576 - val_mrcnn_mask_loss: 0.2468
Epoch 6/50
100/100 [==============================] - 397s 4s/step - loss: 1.3777 - rpn_class_loss: 0.0915 - rpn_bbox_loss: 0.6811 - mrcnn_class_loss: 0.1705 - mrcnn_bbox_loss: 0.1951 - mrcnn_mask_loss: 0.2395 - val_loss: 1.4589 - val_rpn_class_loss: 0.0685 - val_rpn_bbox_loss: 0.7575 - val_mrcnn_class_loss: 0.1578 - val_mrcnn_bbox_loss: 0.2450 - val_mrcnn_mask_loss: 0.2300
Epoch 7/50
 28/100 [=======>......................] - ETA: 4:48 - loss: 1.3326 - rpn_class_loss: 0.0878 - rpn_bbox_loss: 0.6455 - mrcnn_class_loss: 0.1767 - mrcnn_bbox_loss: 0.1880 - mrcnn_mask_loss: 0.2346

...

100/100 [==============================] - 408s 4s/step - loss: 0.8623 - rpn_class_loss: 0.0378 - rpn_bbox_loss: 0.3289 - mrcnn_class_loss: 0.1421 - mrcnn_bbox_loss: 0.1389 - mrcnn_mask_loss: 0.2146 - val_loss: 1.1098 - val_rpn_class_loss: 0.0477 - val_rpn_bbox_loss: 0.4619 - val_mrcnn_class_loss: 0.1549 - val_mrcnn_bbox_loss: 0.2042 - val_mrcnn_mask_loss: 0.2411
Epoch 42/50
100/100 [==============================] - 410s 4s/step - loss: 0.8587 - rpn_class_loss: 0.0387 - rpn_bbox_loss: 0.3195 - mrcnn_class_loss: 0.1447 - mrcnn_bbox_loss: 0.1389 - mrcnn_mask_loss: 0.2170 - val_loss: 1.0479 - val_rpn_class_loss: 0.0476 - val_rpn_bbox_loss: 0.4590 - val_mrcnn_class_loss: 0.1428 - val_mrcnn_bbox_loss: 0.1723 - val_mrcnn_mask_loss: 0.2262
Epoch 43/50
 28/100 [=======>......................] - ETA: 4:55 - loss: 0.8782 - rpn_class_loss: 0.0378 - rpn_bbox_loss: 0.3453 - mrcnn_class_loss: 0.1515 - mrcnn_bbox_loss: 0.1342 - mrcnn_mask_loss: 0.2094

100/100 [==============================] - 413s 4s/step - loss: 0.8727 - rpn_class_loss: 0.0368 - rpn_bbox_loss: 0.3299 - mrcnn_class_loss: 0.1514 - mrcnn_bbox_loss: 0.1390 - mrcnn_mask_loss: 0.2156 - val_loss: 1.0513 - val_rpn_class_loss: 0.0468 - val_rpn_bbox_loss: 0.4570 - val_mrcnn_class_loss: 0.1078 - val_mrcnn_bbox_loss: 0.2043 - val_mrcnn_mask_loss: 0.2355
Epoch 44/50
100/100 [==============================] - 448s 4s/step - loss: 0.8402 - rpn_class_loss: 0.0386 - rpn_bbox_loss: 0.3127 - mrcnn_class_loss: 0.1452 - mrcnn_bbox_loss: 0.1321 - mrcnn_mask_loss: 0.2117 - val_loss: 1.0572 - val_rpn_class_loss: 0.0472 - val_rpn_bbox_loss: 0.4552 - val_mrcnn_class_loss: 0.1317 - val_mrcnn_bbox_loss: 0.1994 - val_mrcnn_mask_loss: 0.2236
Epoch 45/50
 28/100 [=======>......................] - ETA: 5:12 - loss: 0.8318 - rpn_class_loss: 0.0366 - rpn_bbox_loss: 0.3078 - mrcnn_class_loss: 0.1486 - mrcnn_bbox_loss: 0.1288 - mrcnn_mask_loss: 0.2100

100/100 [==============================] - 443s 4s/step - loss: 0.8357 - rpn_class_loss: 0.0369 - rpn_bbox_loss: 0.3141 - mrcnn_class_loss: 0.1394 - mrcnn_bbox_loss: 0.1325 - mrcnn_mask_loss: 0.2128 - val_loss: 1.0010 - val_rpn_class_loss: 0.0473 - val_rpn_bbox_loss: 0.4528 - val_mrcnn_class_loss: 0.1378 - val_mrcnn_bbox_loss: 0.1525 - val_mrcnn_mask_loss: 0.2106
Epoch 46/50
100/100 [==============================] - 445s 4s/step - loss: 0.8522 - rpn_class_loss: 0.0368 - rpn_bbox_loss: 0.3190 - mrcnn_class_loss: 0.1471 - mrcnn_bbox_loss: 0.1374 - mrcnn_mask_loss: 0.2119 - val_loss: 1.1108 - val_rpn_class_loss: 0.0473 - val_rpn_bbox_loss: 0.4510 - val_mrcnn_class_loss: 0.1379 - val_mrcnn_bbox_loss: 0.2196 - val_mrcnn_mask_loss: 0.2550
Epoch 47/50
 28/100 [=======>......................] - ETA: 6:00 - loss: 0.8490 - rpn_class_loss: 0.0375 - rpn_bbox_loss: 0.3119 - mrcnn_class_loss: 0.1520 - mrcnn_bbox_loss: 0.1374 - mrcnn_mask_loss: 0.2102

100/100 [==============================] - 465s 5s/step - loss: 0.8417 - rpn_class_loss: 0.0364 - rpn_bbox_loss: 0.3138 - mrcnn_class_loss: 0.1481 - mrcnn_bbox_loss: 0.1332 - mrcnn_mask_loss: 0.2101 - val_loss: 0.9965 - val_rpn_class_loss: 0.0469 - val_rpn_bbox_loss: 0.4486 - val_mrcnn_class_loss: 0.1175 - val_mrcnn_bbox_loss: 0.1558 - val_mrcnn_mask_loss: 0.2278
Epoch 48/50
100/100 [==============================] - 421s 4s/step - loss: 0.8306 - rpn_class_loss: 0.0362 - rpn_bbox_loss: 0.3087 - mrcnn_class_loss: 0.1395 - mrcnn_bbox_loss: 0.1335 - mrcnn_mask_loss: 0.2127 - val_loss: 1.1153 - val_rpn_class_loss: 0.0463 - val_rpn_bbox_loss: 0.4475 - val_mrcnn_class_loss: 0.1460 - val_mrcnn_bbox_loss: 0.2124 - val_mrcnn_mask_loss: 0.2631
Epoch 49/50
 28/100 [=======>......................] - ETA: 5:27 - loss: 0.8449 - rpn_class_loss: 0.0381 - rpn_bbox_loss: 0.3121 - mrcnn_class_loss: 0.1475 - mrcnn_bbox_loss: 0.1342 - mrcnn_mask_loss: 0.2130

100/100 [==============================] - 444s 4s/step - loss: 0.8473 - rpn_class_loss: 0.0369 - rpn_bbox_loss: 0.3098 - mrcnn_class_loss: 0.1487 - mrcnn_bbox_loss: 0.1369 - mrcnn_mask_loss: 0.2149 - val_loss: 1.0692 - val_rpn_class_loss: 0.0468 - val_rpn_bbox_loss: 0.4455 - val_mrcnn_class_loss: 0.1283 - val_mrcnn_bbox_loss: 0.1842 - val_mrcnn_mask_loss: 0.2644
Epoch 50/50
100/100 [==============================] - 445s 4s/step - loss: 0.8348 - rpn_class_loss: 0.0347 - rpn_bbox_loss: 0.3092 - mrcnn_class_loss: 0.1478 - mrcnn_bbox_loss: 0.1318 - mrcnn_mask_loss: 0.2114 - val_loss: 1.0829 - val_rpn_class_loss: 0.0458 - val_rpn_bbox_loss: 0.4417 - val_mrcnn_class_loss: 0.1269 - val_mrcnn_bbox_loss: 0.2100 - val_mrcnn_mask_loss: 0.2585


In [0]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes.h5")
model.keras_model.save_weights(model_path)

In [13]:
# Detection
class InferenceConfig(KaggleConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

Loading weights from  /content/logs/shapes20180328T0118/mask_rcnn_shapes_0050.h5


In [0]:
# # Test on a random image
# image_id = random.choice(dataset_val.image_ids)

# original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#     modellib.load_image_gt(dataset_val, inference_config, 
#                            image_id, use_mini_mask=False)

# # Ground truth
# log("original_image", original_image)
# log("image_meta", image_meta)
# log("gt_class_id", gt_class_id)
# log("gt_bbox", gt_bbox)
# log("gt_mask", gt_mask)

# visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
#                             dataset_train.class_names, figsize=(8, 8))

In [0]:
# results = model.detect([original_image], verbose=1)

# r = results[0]
# visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
#                             dataset_val.class_names, r['scores'], ax=get_ax())

# # r['masks'].shape = 256*256*num
# # plt.imshow(r['masks'][:,:,0])

In [0]:
# # Compute VOC-Style mAP @ IoU=0.5
# # Running on 10 images. Increase for better accuracy.
# image_ids = np.random.choice(dataset_val.image_ids, 10)
# APs = []
# for image_id in image_ids:
#     # Load image and ground truth data
#     image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#         modellib.load_image_gt(dataset_val, inference_config,
#                                image_id, use_mini_mask=False)
#     molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
#     # Run object detection
#     results = model.detect([image], verbose=0)
#     r = results[0]
#     # Compute AP
#     AP, precisions, recalls, overlaps =\
#         utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
#                          r["rois"], r["class_ids"], r["scores"], r['masks'])
#     APs.append(AP)
    
# print("mAP: ", np.mean(APs))

In [0]:
# rle method-1
# Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)
        
# Iterate over the test IDs and generate run-length encodings for each seperate mask identified by skimage
def mask_to_rle(img_id, mask, test_ids, rles):
    rle = list(prob_to_rles(mask))
    rles.extend(rle)
    test_ids.extend([img_id] * len(rle))
    return test_ids,rles

In [17]:
test_ids = []
rles = []

for image_id in dataset_test.image_ids:
    img_id = dataset_test.img_ids[image_id]
    image = dataset_test.load_image(image_id)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    mask = r['masks'].max(axis=2)
    test_ids, rles = mask_to_rle(img_id, mask, test_ids, rles)

saved_file = 'rcnn-1.csv'    

print (len(list(set(test_ids))))

# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv(saved_file, index=False)

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a file.
uploaded = drive.CreateFile({'title': saved_file})
uploaded.SetContentFile(saved_file)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))
# 4/AACG6-C0U8VukJ2_YnpV2e6QZ59968sOM1sdmryFRHG8oWfN3oS9DDE

65
Uploaded file with ID 1ZbxrpKzPo0uzD7wSWzj3NOK0GXuZOC2R


In [0]:
# rle method-2
def numpy2encoding_no_overlap2(predicts, img_id):
    sum_predicts = np.sum(predicts, axis=2)
    rows, cols = np.where(sum_predicts>=2)
    
    for i in zip(rows, cols):
        instance_indicies = np.where(np.any(predicts[i[0],i[1],:]))[0]
        highest = instance_indicies[0]
        predicts[i[0],i[1],:] = predicts[i[0],i[1],:]*0
        predicts[i[0],i[1],highest] = 1
    
    ImageId = []
    EncodedPixels = []
    for i in range(predicts.shape[2]): 
        rle = run_length_encoding(predicts[:,:,i])
        if len(rle)>0:
            ImageId.append(img_id)
            EncodedPixels.append(rle)    
    return ImageId, EncodedPixels

def run_length_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    run_lengths = ' '.join([str(r) for r in run_lengths])
    return run_lengths

In [19]:
ImageId = []
EncodedPixels = []

for image_id in dataset_test.image_ids:
    img_id = dataset_test.img_ids[image_id]
    image = dataset_test.load_image(image_id)
    results = model.detect([image], verbose=0)
    r = results[0]
#     idx = utils.non_max_suppression(r['rois'], r['scores'], 0.2)
#     ImageId_batch, EncodedPixels_batch = numpy2encoding_no_overlap2(r['masks'][idx], img_id)
    ImageId_batch, EncodedPixels_batch = numpy2encoding_no_overlap2(r['masks'], img_id)
    ImageId += ImageId_batch
    EncodedPixels += EncodedPixels_batch
    
saved_file = 'rcnn-3.csv'    

print (len(list(set(ImageId))))

# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = ImageId
# sub['EncodedPixels'] = pd.Series(EncodedPixels).apply(lambda x: ''.join(str(y) for y in x))
sub['EncodedPixels'] = EncodedPixels
sub.to_csv(saved_file, index=False)

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a file.
uploaded = drive.CreateFile({'title': saved_file})
uploaded.SetContentFile(saved_file)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

65
Uploaded file with ID 1BKLmdohYWLySRC1rCRNBEwk5GaId0tBU


In [21]:
# Save model to google drive
!ls

saved_file = 'mask_rcnn_coco.h5'

# Create & upload a file.
uploaded = drive.CreateFile({'title': saved_file})
uploaded.SetContentFile(saved_file)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

assets		    inspect_model.ipynb    __pycache__	stage1_test
coco.py		    inspect_weights.ipynb  rcnn-1.csv	stage1_test.zip
config.py	    LICENSE		   rcnn-2.csv	stage1_train
datalab		    logs		   rcnn-3.csv	stage1_train.zip
demo.ipynb	    mask_rcnn_coco.h5	   README.md	train_shapes.ipynb
images		    model.py		   samples	utils.py
inspect_data.ipynb  parallel_model.py	   shapes.py	visualize.py
Uploaded file with ID 1G10V30DJq-sTxdG3ihvUBQauoGLxyb9H
